In [1]:
from dotenv import load_dotenv
import os
from agents import Agent, Runner, trace, function_tool
import asyncio
from typing import List, Optional
from IPython.display import display, Markdown
from agents.run_context import RunContextWrapper
from agents.mcp import MCPServerStdio
import chromadb
from chromadb.utils import embedding_functions
from typing import List, Dict

load_dotenv(override=True)

True

In [2]:
def process_chroma_query_result(chroma_result: dict) -> List[Dict[str, str]]:
    """
    Transforms the output of a ChromaDB collection.query() call into a
    list of dictionaries, with each dictionary containing 'id' and 'text'.

    Args:
        chroma_result (dict): The dictionary returned by the collection.query() call.
                              Expected to have 'ids' and 'documents' keys, each
                              containing a nested list.

    Returns:
        list[dict]: A list of dictionaries, where each dictionary has
                    the keys 'id' and 'text'.
    """
    processed_list = []
    ids = chroma_result.get('ids', [[]])[0]
    documents = chroma_result.get('documents', [[]])[0]

    if len(ids) != len(documents):
        print("Warning: Mismatch between number of IDs and documents. Check the query result.")
        return []

    for unique_id, text in zip(ids, documents):
        processed_list.append({"url": unique_id, "text": text})

    return processed_list

In [3]:
@function_tool
def get_headlines(query: str) -> List[Dict[str, str]]:
    """
    Retrieve the top 5 most relevant headlines from the Chroma database
    based on a semantic search using the provided query string.

    Args:
        query (str): The natural language query to search against stored headlines.

    Returns:
        List[Dict[str, str]]: A list of the top 5 matching headlines, where each
                              headline is represented as {'id': str, 'text': str}.
    """

    # Initialize embedding function
    ef = embedding_functions.OpenAIEmbeddingFunction(
        api_key=os.getenv("OPENAI_API_KEY"),
        model_name="text-embedding-3-large"
    )

    # Connect to Chroma persistent client
    chroma_client = chromadb.PersistentClient()

    # Load the headlines collection with embeddings
    headlines = chroma_client.get_collection(
        name="headlines",
        embedding_function=ef
    )

    # Query the collection
    results = headlines.query(
        query_texts=[query],
        n_results=5
    )

    # Process results into clean structured list
    return process_chroma_query_result(results)

In [4]:
instructions = """
You are a News Retrieval Agent designed to use Retrieval-Augmented Generation (RAG) to provide users with relevant news headlines based on their expressed preferences.

Your Responsibilities:
1. Understand the User's Preferences:
   - The user will describe the kinds of news they are interested in.
   - Extract key topics, categories, or entities from the user's request.

2. Formulate Queries for Retrieval:
   - For each distinct topic or entity, generate one or more concise natural-language queries that best represent the user's interest.
   - Example: If the user says "I want to hear about Google and artificial intelligence," generate queries like "Google AI", "Google artificial intelligence", "AI research at Google".

3. Call the Retrieval Tool:
   - Use the function tool get_headlines(query: str) to retrieve candidate news articles.
   - Always request multiple queries if the user has multiple interests.
   - Expect up to 5 headlines per query, structured as {"id": str, "text": str}.

4. Filter and Rerank Results:
   - Carefully judge each headline's relevance only in the context of the user's stated interests.
   - Discard irrelevant or weakly related headlines.
   - Optionally rerank or cluster results if several headlines overlap.

5. Produce the Final Output:
   - Return only the filtered set of headlines that are highly relevant to the user's query.
   - Each headline should be listed clearly (e.g., as a bullet point).
   - Do not fabricate headlines — only return those retrieved from the tool.
   - If no relevant results are found, state that clearly.

Constraints:
- Always rely on get_headlines for retrieval. Do not generate news directly.
- Filter rigorously to avoid irrelevant content.
- Prioritize accuracy and user alignment over quantity of headlines.

Example:
User Prompt: "I'd like to hear about space exploration and Elon Musk."

Agent Steps:
- Extract interests: "space exploration", "Elon Musk".
- Generate queries: "space exploration", "NASA space missions", "Elon Musk news", "SpaceX updates".
- Call get_headlines with each query.
- Retrieve results and filter out irrelevant items.

Final Output:
- NASA Announces New Timeline for Artemis Moon Mission
- SpaceX Successfully Launches Starship Prototype
- Elon Musk Teases Mars Colonization Plans
"""

In [5]:
from pydantic import BaseModel, Field

class Headline(BaseModel):
    """
    A single unique news headline.
    Each headline must be distinct across all categories.
    Do not repeat the same article URL or text in multiple places.
    """
    url: str = Field(
        description="The source URL of the news article. Must be unique across all headlines."
    )
    text: str = Field(
        description="The headline text of the news article. Must be unique across all headlines."
    )


class HeadlinesCategory(BaseModel):
    """
    A group of headlines that belong to a single topical category.
    Categories are only created when the user clearly expresses two or more distinct interests.
    Closely related interests (e.g., 'AI' and 'LLMs') should be merged into one category.
    Different interests (e.g., 'AI' vs. 'Trump tariffs') should be separated into different categories.
    """
    category: str = Field(
        description="The name of the category representing one distinct area of user interest."
    )
    headlines: List[Headline] = Field(
        description="A list of unique headlines relevant to this category. No duplicates within this list or across categories."
    )


class HeadlinesOutput(BaseModel):
    """
    The final structured output of relevant news headlines grouped by category.
    
    Rules:
    - Each headline must be unique across all categories (no duplicate URLs or texts).
    - Categories should only be created if the user has specified two or more distinct interests.
      - Merge closely related interests into one category.
      - Separate unrelated interests into different categories.
    - If the user has only one interest, return a single category with all unique headlines.
    """
    headlines: List[HeadlinesCategory] = Field(
        description="A list of categories containing unique, relevant news headlines. No duplicates across categories."
    )


In [6]:
rag_agent = Agent(
    name="RAG Agent",
    tools=[get_headlines],
    instructions=instructions,
    model="gpt-4.1",
    output_type=HeadlinesOutput
)

In [7]:
prompt = "Show me news on AI and on what Google is up to. Also include any news about Trump and the economy."

In [10]:
with trace("News Semantic Search"):
    result = await Runner.run(rag_agent, input=prompt, max_turns=20)
    structured = result.final_output
    for category in structured.headlines:
        i = 1
        for headline in category.headlines:
            print(f"{i}. {headline.text.replace("\n", "")}: {headline.url}")
            i += 1

1. Google's Gemini gets teased in latest Pixel 10 photography clip - Android CentralThe company's AI is all over its devices, and it's on the way to make capturing memories even easier.: https://www.androidcentral.com/phones/google-pixel/google-pixel-10-series-teaser-gemini-photography-features
1. The Pixel 9 Pro Fold is $600 off ahead of the new model’s debut - The VergeOther deals include a $300 discount on Lenovo’s flawed yet capable Legion Go S, three for $30 4K Blu-rays, and more.: https://www.theverge.com/tech/760131/google-pixel-9-pro-fold-lenovo-legion-go-s-deal-sale
1. "Utterly unqualified": Trump BLS pick gets panned by conservative economists - AxiosConservative economists cited examples of Trump's BLS pick appearing to misunderstand data he would be responsible for.: https://www.axios.com/2025/08/12/trump-bls-ej-antoni-economists
2. Switzerland warns its companies that no, they can't dodge Trump's tariffs by routing goods through the tiny neighboring country of Liechtenstei